In [1]:
import tweepy
import json
from datetime import datetime

In [13]:
API_Key = "i3BgiDWeRDGgx1MieH5pfsQdI"

API_Secret = "W3tQRuhqBhYCm2PSezofEb0N069DNOT2r36LZzdAOQW29FdWwy"

token = None #'616379578-w3ElYEoF40uO4GyW0KoMAnzM0SG9lhiWhVsY0aj3'
secret = None #'nY86GsXMNmttcivMlYJ1iWnqFWhcBsWgLbFbdF5vln9Jz'

In [14]:
if token is not None:
    auth = tweepy.OAuthHandler(token, secret)
    auth = tweepy.
    api = tweepy.API(auth)
else:
    auth = tweepy.OAuthHandler(API_Key, API_Secret)
    try:
        redirect_url = auth.get_authorization_url()
        print(redirect_url)
    except tweepy.TweepError:
        print('Error! Failed to get request token.')
    

https://api.twitter.com/oauth/authorize?oauth_token=wzGqqQAAAAABS6ayAAABe7xdSOY


In [15]:
pin = "2148895"

In [16]:
try:
    auth.get_access_token(pin)
except tweepy.TweepError:
    print('Error! Failed to get access token.')

In [17]:
print("token: " + auth.access_token )
print("secret: " + auth.access_token_secret)

token: 616379578-w3ElYEoF40uO4GyW0KoMAnzM0SG9lhiWhVsY0aj3
secret: nY86GsXMNmttcivMlYJ1iWnqFWhcBsWgLbFbdF5vln9Jz


In [18]:
api = tweepy.API(auth)

In [19]:
import re

In [20]:
pattern = "RT @.*:"
prog = re.compile(pattern)

In [21]:
def getTweets(subject):
    #results = api.search_30_day("BootCamp","mn state fair")
    
    global jsonData 
    jsonData = api.search(subject,count = 100,lang="en")
    dataset = []
    for r in jsonData:
        name  = r._json['user']['screen_name']
        createDate = r._json['created_at']
        retweet = r._json["retweeted"]
        if not retweet:
            try:
                tweet = r._json['extended_tweet']['full_text']
            except:
                tweet = r._json['text']
        if prog.search(tweet) is not None:
            continue
        else:
            print(tweet)
            data = f"{name}, {createDate}, {retweet}, {tweet}"
            dataset.append(data)
    filename = f"datasets/dataset {datetime.now().strftime('%Y%m%d%H_%M_%S')}.txt"
    textfile = open(filename, "w")
    for element in dataset:
        textfile.write(element + "\n")
    textfile.close()
    return dataset


In [22]:
results = getTweets("#biden")

Monotonectally Mesh Intuitive Manufactured #Products with #domains 
💰 #biden #trump Buy #domain… https://t.co/Q1AaRSp3ll
@TheEconomist Good it's such a wonderful time for sister #Biden
Good people in power .
Wow 
#SavePanjshir
#USA 🇺🇸

AP FACT CHECK: The @AP fact checks #Biden's shaky claim of US readiness in #Afghanistan &amp; his statements o… https://t.co/wXsJZZ6u7G
#State department is stopping flights for Americans out of Afghanistan and interpreters? Why?
#BIDEN IS NOT VETTING… https://t.co/HWWwvqv5Gx
ALL #Corrupt @USCongress #FakeNews #MSM #TERRIFIED #WeThePeople #WORLD N #Jeopardy DO NOT #Comply #facebook… https://t.co/m2pBTfes17
"Biden directs federal aid to NY, NJ after deadly flooding" https://t.co/ZTCTk2DeMd #Biden #flooding #federalaid
#Maddow gets paid millions as #Biden's modern day Tokyo Rose. https://t.co/HIcbv0fBru
@sirDukeDevin Of course they would &amp; becomes Republicans talking points!Anything to second guess or undermine… https://t.co/XYKq2Ipnd5
#Biden #Afghanis

In [23]:
print(len(results))
for r in results:
    print(f"** {r}\n")

52
** emojizedcom, Mon Sep 06 18:29:15 +0000 2021, False, Monotonectally Mesh Intuitive Manufactured #Products with #domains 
💰 #biden #trump Buy #domain… https://t.co/Q1AaRSp3ll

** global_police, Mon Sep 06 18:29:09 +0000 2021, False, @TheEconomist Good it's such a wonderful time for sister #Biden
Good people in power .
Wow 
#SavePanjshir

** CurtisSChin, Mon Sep 06 18:28:52 +0000 2021, False, #USA 🇺🇸

AP FACT CHECK: The @AP fact checks #Biden's shaky claim of US readiness in #Afghanistan &amp; his statements o… https://t.co/wXsJZZ6u7G

** patriotlady76, Mon Sep 06 18:27:45 +0000 2021, False, #State department is stopping flights for Americans out of Afghanistan and interpreters? Why?
#BIDEN IS NOT VETTING… https://t.co/HWWwvqv5Gx

** two001snake, Mon Sep 06 18:26:37 +0000 2021, False, ALL #Corrupt @USCongress #FakeNews #MSM #TERRIFIED #WeThePeople #WORLD N #Jeopardy DO NOT #Comply #facebook… https://t.co/m2pBTfes17

** JoanneSpruceC21, Mon Sep 06 18:25:16 +0000 2021, False, "Biden d

In [24]:
test = results[4]

In [25]:
test

'two001snake, Mon Sep 06 18:26:37 +0000 2021, False, ALL #Corrupt @USCongress #FakeNews #MSM #TERRIFIED #WeThePeople #WORLD N #Jeopardy DO NOT #Comply #facebook… https://t.co/m2pBTfes17'

In [26]:
test = test.replace("\n", "")

In [27]:
test

'two001snake, Mon Sep 06 18:26:37 +0000 2021, False, ALL #Corrupt @USCongress #FakeNews #MSM #TERRIFIED #WeThePeople #WORLD N #Jeopardy DO NOT #Comply #facebook… https://t.co/m2pBTfes17'

In [28]:
from textblob import TextBlob

In [29]:
analysis = TextBlob("your mother is nice").sentiment
print(analysis)

Sentiment(polarity=0.6, subjectivity=1.0)


In [50]:
test

'jusdax, Sat Sep 04 15:21:02 +0000 2021, False, if you have the ability to love . Love yourself first.#adityabruh #adityaforever #jusdaxbruh #cheekfiller #fillers… https://t.co/c4zY0ByqFf'